## Сравнение и выбор лучшей модели

Во время решения задачи вы скорее всего будете пробовать разные алгоритмы и обучите достаточно большое количество моделей, подбирая оптимальные гиперпараметры.

Как вы уже знаете, для подбора гиперпараметров есть полу-автоматические методы, а для того, чтобы быть уверенным в хорошем перформансе модели, следует применять кросс-валидацию

Напишем функцию, использующую `KFold` для валидации модели

Но для начала поставим задачу и загрузим данные.

Необходимо по характеристикам дома оценить его цену. **Задача какого типа перед нами стоит?**

Описание данных: https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset

In [1]:
# Загрузим датасет
from sklearn.datasets import fetch_california_housing
import pandas as pd

X, y = fetch_california_housing(return_X_y = True)

X_df = pd.DataFrame(X)
X_df.head()

,0,1,2,3,4,5,6,7
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [2]:
y.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

**Какую метрику бы вы использовали для оценки качества модели?**

Функция кросс-валидации работает следующим образом:
-    Создаем KFold для данных
-    Для каждого фолда обучаем модель и делаем предсказание
-    Считаем метрики для этого фолда

In [5]:
# Импортируем функцию KFold
from sklearn.model_selection import KFold

# Я буду использовать MAE, как наиболее наглядную
from sklearn.metrics import mean_absolute_error

def cross_validation(model, X, y, random_state = 10):
    
    # Массив, в котором будем хранить все скоры
    scores = []

    # Создаем генератор KFold
    kf = KFold(random_state = random_state, shuffle=True)
    # Счетчик итераций
    iter_num = 1

    # Для каждой пары сгенерированных выборок обучаем модель и смотрим не MAE
    for train_index, test_index in kf.split(X):
        print(f'Iteration {iter_num}:')

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Обучаем модель
        model.fit(X_train, y_train)

        pred = model.predict(X_test)

        print(f'MAE fold {iter_num}:', mean_absolute_error(pred, y_test))

        iter_num += 1
        
        scores.append(mean_absolute_error(pred, y_test))
    print("Mean overall:", sum(scores) / len(scores))
    return

Обучим линейную регрессию

In [6]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

cross_validation(lin_reg, X, y)

Iteration 1:
MAE fold 1: 0.5366527228154274
Iteration 2:
MAE fold 2: 0.5283047156817452
Iteration 3:
MAE fold 3: 0.5298611439975122
Iteration 4:
MAE fold 4: 0.5294180890954976
Iteration 5:
MAE fold 5: 0.5346103468564891
Mean overall: 0.5317694036893343


Тот же функционал предоставляет функция `cross_validate` модуля `model_selection`

In [7]:
from sklearn.model_selection import cross_validate

cv = cross_validate(lin_reg, X, y, cv = 5)

print(cv['test_score'])

print("Среднее отколнение:", cv['test_score'].mean())

[0.54866323 0.46820691 0.55078434 0.53698703 0.66051406]
Среднее отколнение: 0.5530311140279224


Чаще всего, в реальном мире модель выбирается не из-за ее производительности, а из-за других, более важных для бизнеса факторах: скорость, интерпретируемость, возможность параллелизации и т.д.

Но мы сейчас пренебрежем этим и сравним между собой линейную регрессию, и решающие деревья

In [8]:
from sklearn.tree import DecisionTreeRegressor

trees = DecisionTreeRegressor(random_state = 123)

cv_trees = cross_validate(trees, X, y)

print(cv_trees['test_score'].mean())

0.3359663432847595


Как видно, производительность модели на решающих деревьях выше чем у линейной регрессии, но также вы могли заметить, что дерево обучалось дольше